# C3_병원 외래 방문 데이터 분석

> 원본 데이터

In [20]:
import pandas as pd
import numpy as np

patients = pd.DataFrame({
    "patient_id": [1, 2, 3, 4, 5, 6],
    "sex": ["F", "M", "F", "M", "F", "M"],
    "age": [34, 58, np.nan, 45, 72, 29],
    "city": ["Seoul", "Busan", "Seoul", "Incheon", "Busan", "Seoul"]
})

visits = pd.DataFrame({
    "visit_id": [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008],
    "patient_id": [1, 1, 2, 3, 4, 5, 5, 7],
    "dept": ["IM", "ENT", "IM", "DERM", "IM", "ENT", "IM", "IM"],
    "visit_date": pd.to_datetime(["2026-01-02","2026-01-10","2026-01-03","2026-01-04",
                                  "2026-01-11","2026-01-05","2026-01-12","2026-01-08"]),
    "cost": [12000, 18000, 25000, 15000, 22000, 16000, 30000, 21000]
})

---

### 1. 환자 데이터 체크

- 각 행별 결측치 개수를 row_na_cnt 칼럼으로 추가하고
- 각 열별 결측치 개수를 col_na_cnt 로 계산하기

In [21]:
# Hint. isna().sum(axis=...)

patients['row_na_cnt'] = patients.isna().sum(axis=1)
#patients라는 데이터 프레임에 'row_na_cnt'라는 칼럼을 추가하고 그 칼럼의 값은
#isna() - 데이터가 결측치면 True, 아니면 False로 출력한다.
#.sum - 값이 True로 나오면 1로 계산하라
#axis=1 - 행을 작업하라는 뜻 

print(patients)

   patient_id sex   age     city  row_na_cnt
0           1   F  34.0    Seoul           0
1           2   M  58.0    Busan           0
2           3   F   NaN    Seoul           1
3           4   M  45.0  Incheon           0
4           5   F  72.0    Busan           0
5           6   M  29.0    Seoul           0


In [ ]:
col_na_cnt = patients.drop(columns='row_na_cnt').isna().sum(axis=0)

print(col_na_cnt)

patient_id    0
sex           0
age           1
city          0
dtype: int64


---

### 2. 환자별 진료비 요약 및 상위 환자 찾기

- 환자 데이터를 기준으로 환자 데이터와 방문자 데이터를 한 눈에 볼 수 있게 병합하기
- 환자별 총 진료비(sum)와 방문 횟수(count) 구하기

In [28]:
df_merged = pd.merge(patients, visits, how='left', on='patient_id')
#pd.merge = 데이터프레임을 병합하는 함수
#두 데이터프레임에 공통으로 들어가있는 patient_id를 기준으로
#how='left': 왼쪽 테이블(patients)의 모든 행을 유지하며 병합한다.

#left merge를 사용하는 이유는 환자데이터에는 있지만 방문기록없는 id6는 포함하고
#방문기록은 있지만 환자데이터에는 없는 id7을 제외하기 위해


print(df_merged)

   patient_id sex   age     city  row_na_cnt  visit_id  dept visit_date  \
0           1   F  34.0    Seoul           0    1001.0    IM 2026-01-02   
1           1   F  34.0    Seoul           0    1002.0   ENT 2026-01-10   
2           2   M  58.0    Busan           0    1003.0    IM 2026-01-03   
3           3   F   NaN    Seoul           1    1004.0  DERM 2026-01-04   
4           4   M  45.0  Incheon           0    1005.0    IM 2026-01-11   
5           5   F  72.0    Busan           0    1006.0   ENT 2026-01-05   
6           5   F  72.0    Busan           0    1007.0    IM 2026-01-12   
7           6   M  29.0    Seoul           0       NaN   NaN        NaT   

      cost  
0  12000.0  
1  18000.0  
2  25000.0  
3  15000.0  
4  22000.0  
5  16000.0  
6  30000.0  
7      NaN  


In [30]:
patient_summary = df_merged.groupby('patient_id')['cost'].agg(['sum', 'count'])
# patient_id로 그룹을 묶어서, cost 열에 대해 sum(합계)과 count(개수)를 계산한다.


print(patient_summary)

                sum  count
patient_id                
1           30000.0      2
2           25000.0      1
3           15000.0      1
4           22000.0      1
5           46000.0      2
6               0.0      0
